### Create Grafana Dashboard

In [ ]:
from grafwiz import *
import os
import v3io_frames as v3f

def handler(context, event):
    
    raw_stream_url = f"https://{os.getenv('API_GATEWAY')}-{os.getenv('PROJECT')}.{os.getenv('IGZ_NAMESPACE_DOMAIN')}/?shardId={os.getenv('SHARD_ID')}&container={os.getenv('IGZ_CONTAINER')}&stream={os.getenv('RAW_VIDEO_STREAM')}"
    tagged_stream_url = f"https://{os.getenv('API_GATEWAY')}-{os.getenv('PROJECT')}.{os.getenv('IGZ_NAMESPACE_DOMAIN')}/?shardId={os.getenv('SHARD_ID')}&container={os.getenv('IGZ_CONTAINER')}&stream={os.getenv('TAGGED_VIDEO_STREAM')}"
    
    params_js = """{
         from:ctrl.range.from.format('x'),  // x is unix ms timestamp
         to:ctrl.range.to.format('x'), 
         height:ctrl.height,
         now:Date.now(),
         interval: ctrl.template('$__interval'),
         sample: 'Not escaped: $__interval',
         since:ctrl.lastRequestTime
        }"""
    
    dash = Dashboard("Stream Viewer Test", start='now', dataSource='iguazio', refresh='1s')
    
    raw_stream = Ajax(title='Raw Stream',
                      url=raw_stream_url,
                      method="iframe",
                      params_js=params_js)
    
    tagged_stream = Ajax(title='Tagged Stream',
                      url=tagged_stream_url,
                      method="iframe",
                      params_js=params_js)
    
    dash.row([raw_stream, tagged_stream])
    
    return dash.deploy('http://grafana')